In [18]:
# Imports & Paths
import pandas as pd
import numpy as np
from pathlib import Path
import os

# print("Current Working Directory:", os.getcwd())

BASE_DIR = Path(os.getcwd()).resolve().parent
RAW_DIR = BASE_DIR / "data" / "raw"

CRIME_DIR = RAW_DIR / "crime_data"
EDU_DIR = RAW_DIR / "education_data"

# print("BASE_DIR:", BASE_DIR)
# print("RAW_DIR:", RAW_DIR)
# print("CRIME_DIR:", CRIME_DIR)
# print("EDU_DIR:", EDU_DIR)

In [19]:
# Load Crime Data
crime_path = CRIME_DIR / "estimated_crimes_1979_2024.csv"

# print("Reading crime file from:", crime_path)

crime_df = pd.read_csv(crime_path)

print("FBI Crime Data Loaded Successfully!")
print("Shape:", crime_df.shape)

crime_df.head()

FBI Crime Data Loaded Successfully!
Shape: (2388, 15)


,year,state_abbr,state_name,population,violent_crime,homicide,rape_legacy,rape_revised,robbery,aggravated_assault,property_crime,burglary,larceny,motor_vehicle_theft,caveats
0,1979,NaN,NaN,220099000,1208030,21460,76390.0,NaN,480700,629480,11041500,3327700,6601000,1112800,NaN
1,1979,AK,Alaska,406000,1994,54,292.0,NaN,445,1203,23193,5616,15076,2501,NaN
2,1979,AL,Alabama,3769000,15578,496,1037.0,NaN,4127,9918,144372,48517,83791,12064,NaN
3,1979,AR,Arkansas,2180000,7984,198,595.0,NaN,1626,5565,70949,21457,45267,4225,NaN
4,1979,AZ,Arizona,2450000,14528,219,1120.0,NaN,4305,8884,177977,48916,116976,12085,NaN


In [20]:
# Crime Data Structure Profiling
print("=== Crime Data Columns ===")
print(list(crime_df.columns))

print("\n=== Year Range ===")
print(crime_df["year"].min(), "→", crime_df["year"].max())

print("\n=== Number of Unique States ===")
print(crime_df["state_name"].nunique())

print("\n=== First 20 Unique States ===")
print(crime_df["state_name"].dropna().unique()[:20])

print("\n=== Missing State Name Count ===")
print(crime_df["state_name"].isna().sum())

=== Crime Data Columns ===
['year', 'state_abbr', 'state_name', 'population', 'violent_crime', 'homicide', 'rape_legacy', 'rape_revised', 'robbery', 'aggravated_assault', 'property_crime', 'burglary', 'larceny', 'motor_vehicle_theft', 'caveats']

=== Year Range ===
1979 → 2024

=== Number of Unique States ===
103

=== First 20 Unique States ===
['Alaska' 'Alabama' 'Arkansas' 'Arizona' 'California' 'Colorado'
 'Connecticut' 'District of Columbia' 'Delaware' 'Florida' 'Georgia'
 'Hawaii' 'Iowa' 'Idaho' 'Illinois' 'Indiana' 'Kansas' 'Kentucky'
 'Louisiana' 'Massachusetts']

=== Missing State Name Count ===
38


### Crime Data Profiling Summary

- The FBI crime dataset contains **15 columns**, including year, state identifiers, population, and major crime categories (violent crime, homicide, robbery, burglary, larceny, etc.).
- The dataset spans a broad temporal range from **1979 to 2024**, which fully covers the analysis period (2015–2024).
- There are **103 unique values** in `state_name`, exceeding the expected 51 (50 states + DC). This indicates the presence of territories, duplicated entries, or summary rows.
- The first 20 state names appear valid and correctly formatted.
- There are **38 rows with missing `state_name`**, which likely correspond to improperly reported or aggregate entries and will need to be removed during cleaning.

In [21]:
# Missing Values Overview
print("=== Missing Value Count by Column (Top 20) ===")
missing = crime_df.isna().sum().sort_values(ascending=False)
print(missing.head(20))

print("\n=== Percentage Missing by Column ===")
missing_pct = (crime_df.isna().mean() * 100).round(2)
print(missing_pct.sort_values(ascending=False).head(20))

=== Missing Value Count by Column (Top 20) ===
caveats                2316
rape_revised           1768
rape_legacy             412
state_abbr               42
state_name               38
year                      0
population                0
violent_crime             0
homicide                  0
robbery                   0
aggravated_assault        0
property_crime            0
burglary                  0
larceny                   0
motor_vehicle_theft       0
dtype: int64

=== Percentage Missing by Column ===
caveats                96.98
rape_revised           74.04
rape_legacy            17.25
state_abbr              1.76
state_name              1.59
year                    0.00
population              0.00
violent_crime           0.00
homicide                0.00
robbery                 0.00
aggravated_assault      0.00
property_crime          0.00
burglary                0.00
larceny                 0.00
motor_vehicle_theft     0.00
dtype: float64


### Crime Data Missing Value Summary

- The `caveats` column has **96.98% missing values**, indicating it is not useful for analysis and should be removed.
- `rape_revised` shows **74.04% missing**, and `rape_legacy` has **17.25% missing**, reflecting changes in FBI reporting definitions. These fields are inconsistent across years and will not be used as primary variables.
- `state_abbr` (1.76% missing) and `state_name` (1.59% missing) contain a small number of missing entries, likely corresponding to summary or invalid rows that should be removed.
- Core analytical columns (`violent_crime`, `property_crime`, `homicide`, `robbery`, `burglary`, `larceny`, `motor_vehicle_theft`) have **0% missing values**, which supports reliable downstream analysis.


### Crime Data Cleaning Plan

Based on the profiling results, the following cleaning steps will be applied to the FBI crime dataset:

1. **Remove the `caveats` column**  
   - Contains ~97% missing values and provides no analytical value.

2. **Ignore `rape_legacy` and `rape_revised` fields**  
   - Missing values occur due to changes in FBI reporting definitions.
   - Analysis will rely on `violent_crime` instead of individual rape metrics.

3. **Remove rows with missing `state_name`**  
   - 38 rows correspond to summary or invalid entries and should be dropped.

4. **Filter the dataset to the analysis period (2015–2024)**  
   - Ensures temporal consistency with the education data.

5. **Standardize column names**  
   - Example: `state_name` → `state`, `year` → `year` (clean lowercase naming).

6. **Remove non-state territories**  
   - Exclude entries for territories such as Guam or Puerto Rico to keep only the 50 states + DC.

